In [1]:
%load_ext autoreload
%autoreload 2

import langfun as lf
from typing import List, Dict

from radar.evaluate.baselines import code_agent
from radar.data import load_task_instances_hf

tasks, df_stats = load_task_instances_hf(split="tasks") # can also specify a different split
print(f"Number of tasks: {len(tasks)}")
df_stats.head(2)

/projects/bdata/kenqgu/Research/Year4/radar_release/RADAR/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of tasks: 313


,task_id,query,answer,artifact_type,artifact_scope,query_cols,artifact_reasoning_cols,table,num_rows,num_cols,recovered_tables_transform_spec,base_data_num_tokens,base_data_token_bucket,perturbation_note
0,ultra-trail-races-rank,We have a dataset of ultra trail running race ...,40.48,outliers,single-column,"[race_year_id, rank, age]",[age],"{'headers': ['race_year_id', 'race', 'rank', '...",113,10,"{'drop_rows': [[]], 'overwrite_cells': [[]]}",7995,8000,Introduced a obvious outliers in age column. S...
1,ultra-trail-races-rank,We have a dataset of ultra trail running race ...,40.13,outliers,single-column,"[race_year_id, rank, age]",[rank],"{'headers': ['race_year_id', 'race', 'rank', '...",113,10,"{'drop_rows': [[]], 'overwrite_cells': [[{'row...",7995,8000,Introduced formatting inconsistencies in rank ...


In [ ]:
def llm_func(prompt_messages: List[Dict[str, str]]):
    llm: lf.LanguageModel = lf.LanguageModel.get("gpt-4o", 
                                                 api_key="",
                                                 debug=True)
    stop_tokens = ["```\n```yaml", "\nDISCUSSION"]
    llm.sampling_options.stop = stop_tokens
    lf_messages = [lf.Message.from_value(m, format="openai") for m in prompt_messages]
    response = lf.query(lf_messages, lm=llm, returns_message=True)
    return response.text, response.sym_jsonify()["metadata"]

task_res = code_agent.run_code_agent(tasks[1], llm_func)

[0] LM INFO (total 7320 tokens):
OpenAI(sampling_options=LMSamplingOptions(temperature=None, max_tokens=None, n=1, top_k=40, top_p=None, stop=[0: '```\n```yaml', 1: '\nDISCUSSION'], random_seed=None, logprobs=False, top_logprobs=None, max_thinking_tokens=None, reasoning_effort=None), cache=None, max_concurrency=None, timeout=120.0, max_attempts=5, retry_interval=(5, 60), exponential_backoff=True, max_retry_interval=300, debug=True, api_endpoint='https://api.openai.com/v1/chat/completions', model='gpt-4o', headers=None, api_key='sk-proj-xSnB4gzcMgSPxxEFW3q7T3BlbkFJLsYqQ384E9CRyuzrwTI4', organization=None, project=None)

[0] PROMPT SENT TO LM (6802 tokens):
`[SystemMessage(text='\n  SETTING: You are an expert-level data scientist. Your job is to answer a data driven question   in rigorous manner given a data table.\n  In your analysis:\n  * Carefully address\n  1) missing data: empty or null entries simulating incomplete information\n  2) bad values: clearly erroneous or placeholder entr

In [6]:
task_res.model_dump()

{'baseline': 'code_agent',
 'task_instance_id': 'tid=ultra-trail-races-rank__artifact-type=outliers__ncols=10__token-bucket=8000',
 'llm_extracted_answer': '40.48',
 'ground_truth': '40.13',
 'is_correct': False,
 'llm_messages': [{'role': 'system',
   'content': '\n  SETTING: You are an expert-level data scientist. Your job is to answer a data driven question   in rigorous manner given a data table.\n  In your analysis:\n  * Carefully address\n  1) missing data: empty or null entries simulating incomplete information\n  2) bad values: clearly erroneous or placeholder entries (e.g., `-1`, `9999`, `TEST`, `#REF!` etc.)\n  3) outliers: implausible extreme values that distort analysis (e.g., 220 breathing rate per minute)\n  4) inconsistent formatting: variations in representing the same value (e.g., `22 lbs`, `22 pounds`, `weight = 22`)\n  5) inconsistent logic: cross-field contradictions violating common-sense logic (e.g., end time before start time)\n  * Attempt to safely recover or co